In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import json
import copy
import scipy.stats as ss


sc = "/home/lukasz/Dokumenty/python gieldowy/WIG/"

pliki = os.listdir(sc + "wszystko_3M")

df = pd.read_csv(sc + "wszystko_3M/" + pliki[0]).set_index("Unnamed: 0").drop(["ddRoznica"], axis=1)

wlasnosci = list(df)

In [15]:
wszystko = pd.DataFrame( columns = wlasnosci )
klasy_all = []
for plik in pliki:
    
    df = pd.read_csv(sc + "wszystko_3M/" + plik).set_index("Unnamed: 0").drop(['ddRoznica'], axis=1)
    klasy = list(df["ZKlasy"])
    df.drop(["ZKlasy"], axis=1, inplace=True)
    
    wszystko = pd.concat( [wszystko, df], axis=0)
    klasy_all += klasy

In [24]:
wszystko_ = pd.concat( [wszystko, pd.DataFrame(data = klasy_all, index = wszystko.index)], axis=1)

wszystko_.to_csv(sc + "zagregowane/wszystkie_dane_3M.csv")

In [4]:
wszystko = pd.read_csv(sc + "zagregowane/wszystkie_dane_3M.csv").set_index("Unnamed: 0")

In [6]:

wszystko = wszystko.replace([np.inf, -np.inf], np.nan)

In [10]:
klasy_all = list(wszystko['0.1'])
wszystko.drop(['0.1'], axis=1, inplace=True)

In [21]:
klasy_all = np.asarray(klasy_all)

ind_jedynek = np.where(klasy_all==1)[0]
ind_zer = np.where(klasy_all==0)[0]

DC = []
ANOVA = []
TT = []

for wlasnosc in wlasnosci:
    
    Z_j = copy.copy(wszystko[wlasnosc].iloc[ind_jedynek])
    Z_z = copy.copy(wszystko[wlasnosc].iloc[ind_zer])
    
    hjedynek = np.sum( ~np.isnan(Z_j) )
    hzer = np.sum( ~np.isnan(Z_z) )
    
    D = np.absolute( np.nanmean(Z_j) - np.nanmean(Z_z) )/np.sqrt( (hjedynek*np.nanvar(Z_j, ddof=1)+
                                                                   hzer*np.nanvar(Z_z, ddof=1))/(hjedynek+hzer-2) )
    
    DC.append(D)
    
    Z_j.dropna(axis=0, inplace=True)
    Z_z.dropna(axis=0, inplace=True)
    
    A = ss.f_oneway( Z_j, Z_z)[1]
    ANOVA.append(A)
    
    T = ss.ttest_ind( Z_j, Z_z)[1]
    
    TT.append(T)
    
    

In [33]:
wl_z_danymi = pd.DataFrame( index = wlasnosci, data = {"T-test": TT, "ANOVA": ANOVA})

In [26]:
wl_z_danymi[wl_z_danymi["T-test"] < 0.05]# and wl_z_danymi["ANOVA"] < 0.05]

,ANOVA,T-test
C0,0.024672,0.024672
C1,0.034503,0.034503
C2,0.046368,0.046368
C6,0.033313,0.033313
C7,0.036185,0.036185
C8,0.040796,0.040796
C39,0.041486,0.041486
C40,0.044866,0.044866
C41,0.009656,0.009656
C42,0.007340,0.007340


In [27]:
wl_z_danymi[wl_z_danymi["ANOVA"] < 0.05]

,ANOVA,T-test
C0,0.024672,0.024672
C1,0.034503,0.034503
C2,0.046368,0.046368
C6,0.033313,0.033313
C7,0.036185,0.036185
C8,0.040796,0.040796
C39,0.041486,0.041486
C40,0.044866,0.044866
C41,0.009656,0.009656
C42,0.007340,0.007340


In [36]:
np.allclose( ANOVA, TT)

True

In [41]:
np.any( ANOVA == TT )

False

In [46]:
with open(sc + "zagregowane/wlasnosci_po_anova.txt", "w") as f:

    json.dump( list( wl_z_danymi[ wl_z_danymi["ANOVA"] < 0.05].index), f)